In [31]:
import numpy as np
from keras.datasets import imdb
from keras.models import Sequential, load_model
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.callbacks import ModelCheckpoint

In [19]:
vocabulary = 5000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=vocabulary)
print(X_train.shape,y_train.shape)

(25000,) (25000,)


In [20]:
print(X_train.shape,y_train.shape)
#How is x and y of the same size and same dim
#Maybe x has 25,000 reviews and y has 25,000 labels
print(X_train[0])
print(len(X_train[0]))
print(y_train[0])
print(len(X_train[1]))
print(y_train[1])
print(X_train[:3])

(25000,) (25000,)
[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 2, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 2, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 2, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 2, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 2, 19, 178, 32]
218
1
189
0
[list([1, 14, 22, 16, 43, 530, 973, 1622, 138

In [18]:
# for our model
# Option 1 - we have to construct dictionary and reverse dictionary, then we need sentences as in the gensim model
# Option 2 - Use pre trained model
#option 3 - use Load data function itself
data=[]
data.append(list([10,12,12]))
data.append(list([10,12,12]))
print(data)

[[10, 12, 12], [10, 12, 12]]


In [22]:
max_review_length = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)

In [23]:
data_path='C:/Users/RAHUL/projects/Many_to_one'

In [46]:
# create the model
def build_model(hidden_size=32):
    #embedding_vecor_length = 32
    model = Sequential()
    model.add(Embedding(vocabulary, hidden_size, input_length=max_review_length))
    model.add(LSTM(hidden_size))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    print(model.summary())
    return model
def train_model(model,reload_filename='NULL',save_best_only=True,batch_size=64,num_epochs=10):
    if(save_best_only):
        checkpointer = ModelCheckpoint(filepath=data_path + '/model-{epoch:02d}-{loss:.4f}.hdf5', verbose=1,monitor='val_loss', save_best_only=True, mode='auto')
    else:
        checkpointer = ModelCheckpoint(filepath=data_path + '/model-{epoch:02d}-{loss:.4f}.hdf5', verbose=1)
    #period option in checkpointer -> after how many epochs to save the model
    if(reload_filename!="NULL"):
            model = load_model(data_path + "/" + reload_filename)
    model.fit(X_train, y_train, validation_split=0.1, epochs=num_epochs, batch_size=batch_size,callbacks=[checkpointer])
    model.save(data_path + "final_model.hdf5")
hidden_size=32
rnn_model=build_model()
#add call back to save model after every epoch

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_9 (Embedding)      (None, 500, 32)           160000    
_________________________________________________________________
lstm_9 (LSTM)                (None, 32)                8320      
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 33        
Total params: 168,353
Trainable params: 168,353
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
train_model(model=rnn_model,reload_filename='model-01-0.2775.hdf5',save_best_only=True)

In [ ]:
#if we use a custom generator then we have to use fit generator
#have to take care of online traing in keras too
#so there is no need to splt into validation set, we can validate internally itself

In [49]:
# Final evaluation of the model
def test_model(filename):
    model = load_model(data_path + "/" + filename)
    scores = model.evaluate(X_test, y_test, verbose=1)
    print("Accuracy: %.2f%%" % (scores[1]*100))
filename="model-01-0.2775.hdf5"
test_model(filename)

25000/25000 [==============================] - 48s 2ms/step
Accuracy: 87.96%
